<a href="https://colab.research.google.com/github/Liying1996/DL_Drugs/blob/main/RBM_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.stats import pearsonr
import seaborn as sns
import scipy.stats as stats
import matplotlib_venn 
import random


In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR 
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
import pickle
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.stats import median_absolute_deviation

from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torchvision

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
gdsc = pd.read_csv('gdrive/MyDrive/GDSC2/GDSC2_IC50_after.csv')
all_express = pd.read_csv('gdrive/MyDrive/GDSC2/expression_after.csv')
cosmic_express = pd.read_csv('gdrive/MyDrive/GDSC2/expression_cosmic_after.csv')

In [ ]:
X = all_express.iloc[:,1:]
y = gdsc[gdsc['Drug_ID']=='Alisertib']['Y']

In [ ]:
# Data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

scaler = MinMaxScaler()
new_X_train = scaler.fit_transform(X_train)
new_X_test = scaler.transform(X_test)

train_loader = DataLoader(dataset=torch.from_numpy(new_X_train).float(), batch_size=50)   
test_loader = DataLoader(dataset=torch.from_numpy(new_X_test).float(), batch_size=50)


In [ ]:
import torch


class RBM():

    def __init__(self, num_visible, num_hidden, k, learning_rate=1e-3, momentum_coefficient=0.5, weight_decay=1e-4, use_cuda=True):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.k = k
        self.learning_rate = learning_rate
        self.momentum_coefficient = momentum_coefficient
        self.weight_decay = weight_decay
        self.use_cuda = use_cuda

        self.weights = torch.randn(num_visible, num_hidden) * 0.1
        self.visible_bias = torch.ones(num_visible) * 0.5
        self.hidden_bias = torch.zeros(num_hidden)

        self.weights_momentum = torch.zeros(num_visible, num_hidden)
        self.visible_bias_momentum = torch.zeros(num_visible)
        self.hidden_bias_momentum = torch.zeros(num_hidden)

        if self.use_cuda:
            self.weights = self.weights.cuda()
            self.visible_bias = self.visible_bias.cuda()
            self.hidden_bias = self.hidden_bias.cuda()

            self.weights_momentum = self.weights_momentum.cuda()
            self.visible_bias_momentum = self.visible_bias_momentum.cuda()
            self.hidden_bias_momentum = self.hidden_bias_momentum.cuda()

    def sample_hidden(self, visible_probabilities):
        hidden_activations = torch.matmul(visible_probabilities, self.weights) + self.hidden_bias
        hidden_probabilities = self._sigmoid(hidden_activations)
        return hidden_probabilities

    def sample_visible(self, hidden_probabilities):
        visible_activations = torch.matmul(hidden_probabilities, self.weights.t()) + self.visible_bias
        visible_probabilities = self._sigmoid(visible_activations)
        return visible_probabilities

    def contrastive_divergence(self, input_data):
        # Positive phase
        positive_hidden_probabilities = self.sample_hidden(input_data)
        positive_hidden_activations = (positive_hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()
        positive_associations = torch.matmul(input_data.t(), positive_hidden_activations)

        # Negative phase
        hidden_activations = positive_hidden_activations

        for step in range(self.k):
            visible_probabilities = self.sample_visible(hidden_activations)
            hidden_probabilities = self.sample_hidden(visible_probabilities)
            hidden_activations = (hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()

        negative_visible_probabilities = visible_probabilities
        negative_hidden_probabilities = hidden_probabilities

        negative_associations = torch.matmul(negative_visible_probabilities.t(), negative_hidden_probabilities)

        # Update parameters
        self.weights_momentum *= self.momentum_coefficient
        self.weights_momentum += (positive_associations - negative_associations)

        self.visible_bias_momentum *= self.momentum_coefficient
        self.visible_bias_momentum += torch.sum(input_data - negative_visible_probabilities, dim=0)

        self.hidden_bias_momentum *= self.momentum_coefficient
        self.hidden_bias_momentum += torch.sum(positive_hidden_probabilities - negative_hidden_probabilities, dim=0)

        batch_size = input_data.size(0)

        self.weights += self.weights_momentum * self.learning_rate / batch_size
        self.visible_bias += self.visible_bias_momentum * self.learning_rate / batch_size
        self.hidden_bias += self.hidden_bias_momentum * self.learning_rate / batch_size

        self.weights -= self.weights * self.weight_decay  # L2 weight decay

        # Compute reconstruction error
        error = torch.sum((input_data - negative_visible_probabilities)**2)

        return error

    def _sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))

    def _random_probabilities(self, num):
        random_probabilities = torch.rand(num)

        if self.use_cuda:
            random_probabilities = random_probabilities.cuda()

        return random_probabilities


In [ ]:
print('Training RBM...')

rbm = RBM(17737, 5000, 2, use_cuda=torch.cuda.is_available())

for epoch in range(100):
    epoch_error = 0.0

    for batch in train_loader:

        batch = batch.cuda()
        batch_error = rbm.contrastive_divergence(batch)

        epoch_error += batch_error
    
    if not (epoch % 5): 
        print('Epoch Error (epoch=%d): %.4f' % (epoch, epoch_error))

Training RBM...
Epoch Error (epoch=0): 1527733.6250
Epoch Error (epoch=1): 1159543.6250
Epoch Error (epoch=2): 1146903.7500
Epoch Error (epoch=3): 1139408.3750
Epoch Error (epoch=4): 1131059.2500
Epoch Error (epoch=5): 1124355.3750
Epoch Error (epoch=6): 1116738.1250
Epoch Error (epoch=7): 1109540.3750
Epoch Error (epoch=8): 1100498.2500
Epoch Error (epoch=9): 1093051.6250
Epoch Error (epoch=10): 1084496.8750
Epoch Error (epoch=11): 1075575.0000
Epoch Error (epoch=12): 1065843.8750
Epoch Error (epoch=13): 1056358.6250
Epoch Error (epoch=14): 1045822.0000
Epoch Error (epoch=15): 1034943.9375
Epoch Error (epoch=16): 1024928.1875
Epoch Error (epoch=17): 1009715.0000
Epoch Error (epoch=18): 1000810.1875
Epoch Error (epoch=19): 987963.5625
Epoch Error (epoch=20): 972500.7500
Epoch Error (epoch=21): 958094.0000
Epoch Error (epoch=22): 948794.3750
Epoch Error (epoch=23): 930014.9375
Epoch Error (epoch=24): 913151.4375
Epoch Error (epoch=25): 898779.8125
Epoch Error (epoch=26): 884658.0625
Epo